In [1]:
import numpy
import matplotlib.pyplot as plt
import time

In [2]:
from utils import clustering # cluster_mean为自定义库，包含SFHC、层次聚类算法的函数

## 聚类输入

In [ ]:
max_min_time={5: [119.05935933078378, 33.349999999999994],
 3: [139.5, 46.0],
 43: [85.4417573918544, 23.0],
 44: [42.90484734256568, 23.0],
 46: [123.25, 38.0],
 138: [199.10377691912151, 72.39134340165585],
 34: [98.13854500484459, 25.0],
 32: [254.5, 47.0],
 297: [133.24436954846146, 44.0],
 316: [183.13197969543145, 36.0],
 59: [90.01347911474551, 46.0],
 71: [104.96755728211456, 40.0],
 57: [284.0, 50.0],
 55: [107.75, 27.0],
 153: [180.25, 48.0],
 152: [172.22564912405616, 68.0],
 221: [141.19167274271376, 71.0],
 385: [168.75, 68.0],
 42: [104.48327849376622, 34.0],
  139: [117.25, 36.0]}

In [3]:
info=numpy.load(r'路段信息_简略.npy',allow_pickle=True).item()

In [4]:
mean_mat=numpy.load(r'mean_mat.npy',allow_pickle=True).item()#记录了各路段每时刻在训练集的40天中的平均旅行时间

In [ ]:
# K均值聚类输入
test=list(mean_mat_norm.values())
from sklearn.cluster import KMeans
import time
start=time.time()
kmeans = KMeans(n_clusters=8, random_state=0).fit(data)
end=time.time()
print(end-start)
kmeans.labels_

In [ ]:
segs=[139,316,46,47,138,33,34,45,246,385,42,37,41,43,44,152]
clusters=[6, 5, 7, 7, 7, 1, 0, 5, 3, 4, 0, 2, 6, 2, 7, 0]
cluster_dict={}
for index,seg in enumerate(segs):
    if clusters[index] not in cluster_dict:
        cluster_dict[clusters[index]]=[]
    cluster_dict[clusters[index]].append(seg)
# cluster_dict
print(list(cluster_dict.values()))

谱聚类

In [ ]:
X=clustering.dump(mean_mat)#导出谱聚类的affinity matrix

In [ ]:
from sklearn.cluster import SpectralClustering
clustering = SpectralClustering(n_clusters=8,
                                 assign_labels='discretize',
                                 random_state=0,
                                 affinity='precomputed',verbose=True).fit(X)
clustering.labels_

In [ ]:
segs=[139,316,46,47,138,33,34,45,246,385,42,37,41,43,44,152]
clusters=[4, 2, 0, 0, 1, 1, 2, 1, 7, 3, 6, 4, 6, 5, 6, 5]
cluster_dict={}
for index,seg in enumerate(segs):
    if clusters[index] not in cluster_dict:
        cluster_dict[clusters[index]]=[]
    cluster_dict[clusters[index]].append(seg)
print(list(cluster_dict.values()))

SFHC(自定义层次聚类)和传统层次聚类

In [ ]:
mean_mat_norm={} #进行最大-最小值归一化，从而获得更明显的旅行时间变化趋势
for seg in mean_mat:
    max_val=max(mean_mat[seg])
    min_val=min(mean_mat[seg])
    min_arr=numpy.array([min_val for i in range(len(mean_mat[55]))])
    mean_mat_norm[seg]=list(numpy.around((mean_mat[seg]-min_arr)/(max_val-min_val),2))

In [6]:
mean_mat_slot_20={}
for seg in mean_mat:
    new_record=[numpy.around(numpy.mean(mean_mat[seg][i*4:(i+1)*4]),2) for i in range(len(mean_mat[43])//4)]
    mean_mat_slot_20[seg]=new_record

In [7]:
mean_mat_slot_20_norm={}
for seg in mean_mat_slot_20:
    max_val=max(mean_mat_slot_20[seg])
    min_val=min(mean_mat_slot_20[seg])
    min_arr=numpy.array([min_val for i in range(len(mean_mat[43])//4)])
    mean_mat_slot_20_norm[seg]=list(numpy.around((mean_mat_slot_20[seg]-min_arr)/(max_val-min_val),2))

In [ ]:
start=time.time()
clustering.hierarchical(mean_mat,linkage=1,method=0,threshold=0.65)
end=time.time()
end-start

In [ ]:
start=time.time()
clustering.sfhc(mean_mat,linkage=1,threshold=0.65)
end=time.time()
end-start

In [ ]:
plt.rcParams['figure.figsize']=(10,5)
plt.rcParams['font.sans-serif']=['Microsoft YaHei','SimHei']
for index,cluster in enumerate([[5, 3, 71, 57],[43, 297, 153, 152, 221, 42],[44, 46, 139, 59],[138],[34],[32],[55],[385]]):
    for seg in cluster:
        plt.plot([i for i in range(44)],mean_mat_slot_20_norm[seg],label=info[str(seg)][0])
        plt.xlabel('时间')
        plt.ylabel('归一化的行程时间')
    plt.legend()
    plt.show()

# 自动构建邻接表（待完成）

In [ ]:
basic_info={'adj':[],'body':'','start':'','end':''}
geo_info={}
segs=[5, 3, 43, 44, 46, 138, 34, 32, 297, 316, 59, 71, 57, 55, 153, 152, 221, 385, 42, 139]
for seg in segs:
    geo=info[str(seg)][0]
    geo=geo.replace('（',' ').replace('）',' ').replace('-',' ')
    geo=geo.split(' ')
    geo_info[seg]=basic_info
    geo_info[seg]['body']=geo[0]
    geo_info[seg]['start']=geo[1]
    geo_info[seg]['end']=geo[2]
for seg in segs:
    target=geo_info[seg]
    for each in segs:
        test=geo_info[each]
        if target['end'][:-2]==test['body'][:-2]:
            if target['body'][:-2]==test['start'][:-2]:
                target['adj'].append(each)
    geo_info[seg]=target

In [ ]:
list_1=[[5, 3, 43, 44, 46, 34, 32, 297, 59, 71, 57, 55, 153, 152, 221, 42], [138], [139], [385]]
list_1_set=[set(i) for i in list_1]
list_2=[[5, 3, 43, 34, 32, 297, 59, 71, 57, 55, 153, 152, 221, 42], [44, 46, 139], [138], [385]]
list_2_set=[set(i) for i in list_2]
for each in list_1_set:
    if each not in list_2_set:
        print('not matched')
        exit(0)